In [3]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [66]:
subject = 'Minas Gerais - Consumo de Cimento (t)'
split_index = 203 #Referente aos 230 anos de input  
train_split = split_index + 1 - 12*3

In [50]:
data = pd.read_csv('2003_mo_model_input_MG.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Minas Gerais - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Minas Gerais - value,Minas Gerais - Produção de Cimento (t),Minas Gerais - Desemprego,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado,Minas Gerais - Consumo de Cimento (t)
0,2003-1,0.756537,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,0.334536,648.891160,8.242462,2.937752e+08,1.725077e+07,14.096054,2.759117e+08,247.134
1,2003-2,0.756708,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,0.335670,652.396314,8.236470,2.941480e+08,1.726959e+07,14.102603,2.760648e+08,305.708
2,2003-3,0.756879,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,0.336553,655.911573,8.230479,2.945208e+08,1.728841e+07,14.109152,2.762178e+08,284.168
3,2003-4,0.757050,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,0.336531,656.746430,8.224488,2.948935e+08,1.730723e+07,14.115701,2.763709e+08,286.594
4,2003-5,0.757222,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,0.336406,660.114672,8.218497,2.952663e+08,1.732605e+07,14.122250,2.765239e+08,312.455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.622619,1250.988075,NaN,NaN,NaN,NaN,NaN,633.858
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.621866,1247.030458,NaN,NaN,NaN,NaN,NaN,612.428
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.620866,1244.634022,NaN,NaN,NaN,NaN,NaN,579.831
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.619585,1241.214294,NaN,NaN,NaN,NaN,NaN,559.950


In [151]:
input_data = data.iloc[:split_index + 1,1:-1]
# input_data = data.iloc[:split_index + 1,:]
input_data = (input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)
input_data

,Minas Gerais - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Minas Gerais - value,Minas Gerais - Produção de Cimento (t),Minas Gerais - Desemprego,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado
0,-1.968698,2.782450,4.506880,2.067266,2.574314,-2.064648,-2.469876,3.184489,-1.575817,-2.202855,-0.752424,-1.806910,-1.971018,-3.111816,-2.777792
1,-1.943192,2.407943,4.328460,1.285816,2.334870,-2.037913,-2.431875,3.029073,-1.560482,-2.182122,-0.759292,-1.786501,-1.919572,-3.024730,-2.713279
2,-1.917685,2.179073,4.129086,2.919965,2.221334,-2.011179,-2.393874,2.926505,-1.548550,-2.161328,-0.766160,-1.766091,-1.868126,-2.937643,-2.648766
3,-1.892178,2.077086,3.911409,1.142823,2.040542,-1.984445,-2.355872,2.828220,-1.548848,-2.156390,-0.773028,-1.745682,-1.816680,-2.850557,-2.584252
4,-1.866672,1.942128,3.663912,2.355956,1.912744,-1.957710,-2.317871,3.036493,-1.550544,-2.136466,-0.779896,-1.725272,-1.765234,-2.763471,-2.519739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,1.081278,-0.689886,-0.724085,-1.189161,3.148408,1.378950,0.117681,0.370628,1.208028,1.070020,0.646276,0.920674,-0.608959,-0.023342,0.468241
200,1.064113,-0.441954,-0.736434,-1.296499,3.219670,1.391539,0.092456,0.427775,1.256499,1.058717,0.633557,0.902641,-0.600196,-0.045810,0.445578
201,1.046949,-0.132782,-0.738433,-1.444029,3.421082,1.404128,0.067231,0.538287,1.305916,1.081210,0.620837,0.884609,-0.591433,-0.068278,0.422914
202,1.029785,0.084061,-0.738236,-1.444143,3.373840,1.416717,0.042006,0.666156,1.356297,1.085207,0.608118,0.866576,-0.582670,-0.090746,0.400250


In [69]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      305.708
1      284.168
2      286.594
3      312.455
4      299.651
        ...   
235    612.428
236    579.831
237    559.950
238    559.950
239        NaN
Name: Minas Gerais - Consumo de Cimento (t), Length: 240, dtype: float64

In [152]:
# input para treinamento
train_input = input_data.iloc[:train_split]
train_input

,Minas Gerais - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Minas Gerais - value,Minas Gerais - Produção de Cimento (t),Minas Gerais - Desemprego,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado
0,-1.968698,2.782450,4.506880,2.067266,2.574314,-2.064648,-2.469876,3.184489,-1.575817,-2.202855,-0.752424,-1.806910,-1.971018,-3.111816,-2.777792
1,-1.943192,2.407943,4.328460,1.285816,2.334870,-2.037913,-2.431875,3.029073,-1.560482,-2.182122,-0.759292,-1.786501,-1.919572,-3.024730,-2.713279
2,-1.917685,2.179073,4.129086,2.919965,2.221334,-2.011179,-2.393874,2.926505,-1.548550,-2.161328,-0.766160,-1.766091,-1.868126,-2.937643,-2.648766
3,-1.892178,2.077086,3.911409,1.142823,2.040542,-1.984445,-2.355872,2.828220,-1.548848,-2.156390,-0.773028,-1.745682,-1.816680,-2.850557,-2.584252
4,-1.866672,1.942128,3.663912,2.355956,1.912744,-1.957710,-2.317871,3.036493,-1.550544,-2.136466,-0.779896,-1.725272,-1.765234,-2.763471,-2.519739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,1.334554,-0.742853,-0.619453,0.617009,-0.970725,0.980624,0.764972,-1.322806,0.717612,-0.030629,1.239700,1.136971,-0.986304,0.021589,0.677638
164,1.337445,-0.787367,-0.611176,0.459632,-0.841670,0.991608,0.753473,-1.355523,0.715300,-0.017187,1.195852,1.142243,-0.994112,0.041435,0.688796
165,1.340335,-0.757940,-0.599627,0.493659,-0.838453,1.002592,0.741973,-1.376175,0.716462,-0.003286,1.152004,1.147515,-1.001919,0.061280,0.699954
166,1.343226,-0.917469,-0.589237,0.617024,-1.006001,1.013577,0.730473,-1.414146,0.715461,0.000561,1.108156,1.152788,-1.009727,0.081125,0.711112


In [153]:
# Alvo para treinamento
train_target = target_data.iloc[:train_split]
train_target

0      305.708000
1      284.168000
2      286.594000
3      312.455000
4      299.651000
          ...    
163    411.310457
164    406.131343
165    400.952229
166    395.773114
167    390.594000
Name: Minas Gerais - Consumo de Cimento (t), Length: 168, dtype: float64

In [155]:
#input de test (Ano 2021)
test_input = input_data.iloc[train_split:]
test_input

,Minas Gerais - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Minas Gerais - value,Minas Gerais - Produção de Cimento (t),Minas Gerais - Desemprego,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado
168,1.349007,-1.032458,-0.585256,-0.505289,-1.202612,1.035545,0.707473,-1.446117,0.715008,0.019780,1.020461,1.163332,-1.025342,0.120816,0.733427
169,1.343125,-1.077854,-0.581056,-0.373612,-1.139987,1.046449,0.692399,-1.526274,0.715570,0.029139,1.001834,1.162557,-1.011264,0.132877,0.734965
170,1.337243,-1.036724,-0.574769,-0.291560,-1.053534,1.057352,0.677324,-1.610341,0.716709,0.052997,0.983208,1.161781,-0.997185,0.144939,0.736502
171,1.331361,-1.013400,-0.567021,-0.450118,-0.932974,1.068256,0.662249,-1.601301,0.718059,0.071070,0.964582,1.161006,-0.983107,0.157000,0.738040
172,1.325480,-1.059228,-0.557842,-0.909054,-0.763756,1.079159,0.647175,-1.618284,0.719639,0.140932,0.945955,1.160230,-0.969029,0.169061,0.739577
173,1.319598,-1.170032,-0.548012,0.387442,-0.644652,1.090063,0.632100,-1.602348,0.721466,0.186483,0.927329,1.159455,-0.954950,0.181122,0.741115
174,1.313716,-1.115621,-0.537244,-0.937589,-0.541601,1.100966,0.617026,-1.686659,0.723499,0.258333,0.908702,1.158679,-0.940872,0.193184,0.742652
175,1.307834,-1.182159,-0.524149,-1.212466,-0.565747,1.111870,0.601951,-1.688184,0.725580,0.325035,0.890076,1.157904,-0.926794,0.205245,0.744190
176,1.301952,-1.270623,-0.519647,-0.499856,-0.536256,1.122773,0.586877,-1.692173,0.727713,0.348449,0.871450,1.157128,-0.912715,0.217306,0.745727
177,1.296071,-1.321931,-0.510389,-0.120028,-0.544036,1.133677,0.571802,-1.690760,0.729905,0.378109,0.852823,1.156353,-0.898637,0.229368,0.747265


In [156]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[train_split:split_index + 1]
test_target

168    338.767
169    419.962
170    387.800
171    391.122
172    409.739
173    413.460
174    497.019
175    476.478
176    421.395
177    413.641
178    338.033
179    431.028
180    305.105
181    402.406
182    419.419
183    336.054
184    568.395
185    541.488
186    537.851
187    526.265
188    559.293
189    466.646
190    416.049
191    501.489
192    460.699
193    445.583
194    509.271
195    532.602
196    500.905
197    588.495
198    589.349
199    543.982
200    614.074
201    557.419
202    412.582
203    468.600
Name: Minas Gerais - Consumo de Cimento (t), dtype: float64

In [212]:
def validation_splitter(df, div_factor):
    split_factor = len(df)//div_factor
    positions_to_drop = []
    for i in range(split_factor):
        pos = df.shape[0] - (i*6 + 1)
        positions_to_drop.append(pos)
    
    return df.drop(positions_to_drop), df.iloc[positions_to_drop]

In [216]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=100)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    train, train_val = validation_splitter(train_input, 8)
    target,target_val = validation_splitter(train_target, 8)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val, target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [217]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(20)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][100:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [218]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[3398967367, 1004234861, 3491310526, 3081849017, 285837871, 391143312, 3953714150, 505551795, 4281525531, 1519902545, 1494668786, 3464669967, 335283588, 2188430177, 1838088161, 3758559584, 499089382, 3936169230, 2348349913, 2860368336]


Step: 0 ___________________________________________
val_loss: 3387.944091796875
winner_seed: 3398967367


Step: 1 ___________________________________________
val_loss: 2000.4512939453125
winner_seed: 1004234861


Step: 2 ___________________________________________
val_loss: 2170.27685546875


Step: 3 ___________________________________________
val_loss: 1242.4888916015625
winner_seed: 3081849017


Step: 4 ___________________________________________
val_loss: 1759.31884765625


Step: 5 ___________________________________________
val_loss: 1552.769287109375


Step: 6 ___________________________________________
val_loss: 1611.427978515625


Step: 7 ___________________________________________
val_loss: 1601.8441162109375


Step: 8 _________________________

In [219]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
5/5 [==============================] - 0s 26ms/step - loss: 156252.8438 - val_loss: 299912.7500
Epoch 2/10000
5/5 [==============================] - 0s 5ms/step - loss: 203682.8594 - val_loss: 214028.9062
Epoch 3/10000
5/5 [==============================] - 0s 4ms/step - loss: 179018.4062 - val_loss: 181028.4844
Epoch 4/10000
5/5 [==============================] - 0s 4ms/step - loss: 165276.6875 - val_loss: 165838.2500
Epoch 5/10000
5/5 [==============================] - 0s 4ms/step - loss: 153500.5781 - val_loss: 139483.8438
Epoch 6/10000
5/5 [==============================] - 0s 4ms/step - loss: 133540.5625 - val_loss: 127523.6641
Epoch 7/10000
5/5 [==============================] - 0s 4ms/step - loss: 120212.6562 - val_loss: 136705.3750
Epoch 8/10000
5/5 [==============================] - 0s 5ms/step - loss: 106091.5859 - val_loss: 141170.8125
Epoch 9/10000
5/5 [==============================] - 0s 4ms/step - loss: 97614.1484 - val_loss: 95902.2109
Epoch 10/10000
5/5 [

5/5 [==============================] - 0s 4ms/step - loss: 2782.5774 - val_loss: 3671.6626
Epoch 78/10000
5/5 [==============================] - 0s 5ms/step - loss: 2775.6228 - val_loss: 3270.7903
Epoch 79/10000
5/5 [==============================] - 0s 4ms/step - loss: 3048.0657 - val_loss: 3365.2173
Epoch 80/10000
5/5 [==============================] - 0s 4ms/step - loss: 2792.8027 - val_loss: 3427.6194
Epoch 81/10000
5/5 [==============================] - 0s 4ms/step - loss: 2869.2529 - val_loss: 3571.2932
Epoch 82/10000
5/5 [==============================] - 0s 4ms/step - loss: 2956.7051 - val_loss: 2565.3523
Epoch 83/10000
5/5 [==============================] - 0s 5ms/step - loss: 2766.6614 - val_loss: 2796.8911
Epoch 84/10000
5/5 [==============================] - 0s 4ms/step - loss: 2790.3391 - val_loss: 4089.4077
Epoch 85/10000
5/5 [==============================] - 0s 4ms/step - loss: 2599.5151 - val_loss: 2653.1440
Epoch 86/10000
5/5 [==============================] - 0s 4ms/

Epoch 154/10000
5/5 [==============================] - 0s 5ms/step - loss: 2394.5354 - val_loss: 2265.4841
Epoch 155/10000
5/5 [==============================] - 0s 5ms/step - loss: 2268.6763 - val_loss: 2362.4426
Epoch 156/10000
5/5 [==============================] - 0s 4ms/step - loss: 2408.2812 - val_loss: 2937.0837
Epoch 157/10000
5/5 [==============================] - 0s 5ms/step - loss: 2567.7520 - val_loss: 3071.9448
Epoch 158/10000
5/5 [==============================] - 0s 5ms/step - loss: 2342.7705 - val_loss: 3017.0396
Epoch 159/10000
5/5 [==============================] - 0s 4ms/step - loss: 2300.6780 - val_loss: 2809.8848
Epoch 160/10000
5/5 [==============================] - 0s 4ms/step - loss: 2139.1953 - val_loss: 2803.4851
Epoch 161/10000
5/5 [==============================] - 0s 5ms/step - loss: 2287.6033 - val_loss: 2893.7534
Epoch 162/10000
5/5 [==============================] - 0s 5ms/step - loss: 2134.9976 - val_loss: 2733.7722
Epoch 163/10000
5/5 [================

5/5 [==============================] - 0s 5ms/step - loss: 2117.9663 - val_loss: 2449.0488
Epoch 231/10000
5/5 [==============================] - 0s 4ms/step - loss: 2293.4048 - val_loss: 2312.5144
Epoch 232/10000
5/5 [==============================] - 0s 5ms/step - loss: 2322.8679 - val_loss: 2110.2825
Epoch 233/10000
5/5 [==============================] - 0s 5ms/step - loss: 2111.4250 - val_loss: 2116.5066
Epoch 234/10000
5/5 [==============================] - 0s 5ms/step - loss: 2004.6055 - val_loss: 2126.2927
Epoch 235/10000
5/5 [==============================] - 0s 5ms/step - loss: 1995.6947 - val_loss: 2262.1372
Epoch 236/10000
5/5 [==============================] - 0s 5ms/step - loss: 1995.7142 - val_loss: 2097.7229
Epoch 237/10000
5/5 [==============================] - 0s 5ms/step - loss: 2164.6743 - val_loss: 2139.1104
Epoch 238/10000
5/5 [==============================] - 0s 5ms/step - loss: 2068.3899 - val_loss: 2168.2471
Epoch 239/10000
5/5 [==============================] 

5/5 [==============================] - 0s 5ms/step - loss: 1928.7778 - val_loss: 2325.7935
Epoch 307/10000
5/5 [==============================] - 0s 5ms/step - loss: 1929.1841 - val_loss: 2642.7996
Epoch 308/10000
5/5 [==============================] - 0s 4ms/step - loss: 2011.1178 - val_loss: 2567.9453
Epoch 309/10000
5/5 [==============================] - 0s 5ms/step - loss: 1950.9091 - val_loss: 2832.9670
Epoch 310/10000
5/5 [==============================] - 0s 5ms/step - loss: 1976.2196 - val_loss: 2438.4011
Epoch 311/10000
5/5 [==============================] - 0s 5ms/step - loss: 2113.7480 - val_loss: 2306.8982
Epoch 312/10000
5/5 [==============================] - 0s 5ms/step - loss: 2072.6421 - val_loss: 2234.8799
Epoch 313/10000
5/5 [==============================] - 0s 4ms/step - loss: 2139.2163 - val_loss: 2466.6782
Epoch 314/10000
5/5 [==============================] - 0s 5ms/step - loss: 2069.4673 - val_loss: 2600.5525
Epoch 315/10000
5/5 [==============================] 

...

5/5 [==============================] - 0s 4ms/step - loss: 1720.0292 - val_loss: 1754.7191
Epoch 1447/10000
5/5 [==============================] - 0s 4ms/step - loss: 1677.2397 - val_loss: 1783.7020
Epoch 1448/10000
5/5 [==============================] - 0s 5ms/step - loss: 1674.7922 - val_loss: 1677.6388
Epoch 1449/10000
5/5 [==============================] - 0s 5ms/step - loss: 1718.8265 - val_loss: 1601.6135
Epoch 1450/10000
5/5 [==============================] - 0s 4ms/step - loss: 1718.1266 - val_loss: 1563.5432
Epoch 1451/10000
5/5 [==============================] - 0s 4ms/step - loss: 1730.9847 - val_loss: 1682.7346
Epoch 1452/10000
5/5 [==============================] - 0s 5ms/step - loss: 1905.5948 - val_loss: 1679.3186
Epoch 1453/10000
5/5 [==============================] - 0s 5ms/step - loss: 1931.9519 - val_loss: 1975.9265
Epoch 1454/10000
5/5 [==============================] - 0s 4ms/step - loss: 2036.3234 - val_loss: 1883.3768
Epoch 1455/10000
5/5 [=======================

5/5 [==============================] - 0s 4ms/step - loss: 1879.2498 - val_loss: 1753.2515
Epoch 1523/10000
5/5 [==============================] - 0s 4ms/step - loss: 1908.0837 - val_loss: 2005.1161
Epoch 1524/10000
5/5 [==============================] - 0s 4ms/step - loss: 1732.8738 - val_loss: 1956.8875
Epoch 1525/10000
5/5 [==============================] - 0s 4ms/step - loss: 1813.1560 - val_loss: 1951.9182
Epoch 1526/10000
5/5 [==============================] - 0s 4ms/step - loss: 1844.7587 - val_loss: 1963.6842
Epoch 1527/10000
5/5 [==============================] - 0s 4ms/step - loss: 1922.3053 - val_loss: 1867.0753
Epoch 1528/10000
5/5 [==============================] - 0s 4ms/step - loss: 1829.4966 - val_loss: 1960.8583
Epoch 1529/10000
5/5 [==============================] - 0s 4ms/step - loss: 1974.4905 - val_loss: 2232.8704
Epoch 1530/10000
5/5 [==============================] - 0s 4ms/step - loss: 1750.4673 - val_loss: 2015.3579
Epoch 1531/10000
5/5 [=======================

5/5 [==============================] - 0s 5ms/step - loss: 2077.4458 - val_loss: 2265.5327
Epoch 1599/10000
5/5 [==============================] - 0s 4ms/step - loss: 2286.3975 - val_loss: 2328.8916
Epoch 1600/10000
5/5 [==============================] - 0s 4ms/step - loss: 1980.1433 - val_loss: 2197.1497
Epoch 1601/10000
5/5 [==============================] - 0s 4ms/step - loss: 2004.5143 - val_loss: 2122.2183
Epoch 1602/10000
5/5 [==============================] - 0s 4ms/step - loss: 1901.2434 - val_loss: 1982.5494
Epoch 1603/10000
5/5 [==============================] - 0s 4ms/step - loss: 1877.1082 - val_loss: 2083.8076
Epoch 1604/10000
5/5 [==============================] - 0s 4ms/step - loss: 1796.4625 - val_loss: 2204.0830
Epoch 1605/10000
5/5 [==============================] - 0s 4ms/step - loss: 1784.7632 - val_loss: 2453.8809
Epoch 1606/10000
5/5 [==============================] - 0s 4ms/step - loss: 1864.7219 - val_loss: 2395.8215
Epoch 1607/10000
5/5 [=======================

5/5 [==============================] - 0s 4ms/step - loss: 1711.1962 - val_loss: 2324.8679
Epoch 1675/10000
5/5 [==============================] - 0s 4ms/step - loss: 1675.4054 - val_loss: 2361.7734
Epoch 1676/10000
5/5 [==============================] - 0s 4ms/step - loss: 1826.7672 - val_loss: 2643.5186
Epoch 1677/10000
5/5 [==============================] - 0s 4ms/step - loss: 2004.4111 - val_loss: 2619.1497
Epoch 1678/10000
5/5 [==============================] - 0s 4ms/step - loss: 1816.7686 - val_loss: 2634.8906
Epoch 1679/10000
5/5 [==============================] - 0s 4ms/step - loss: 1653.1024 - val_loss: 2800.4604
Epoch 1680/10000
5/5 [==============================] - 0s 4ms/step - loss: 1849.9009 - val_loss: 2465.5884
Epoch 1681/10000
5/5 [==============================] - 0s 4ms/step - loss: 1911.8862 - val_loss: 2621.3430
Epoch 1682/10000
5/5 [==============================] - 0s 4ms/step - loss: 1705.0448 - val_loss: 2690.8992
Epoch 1683/10000
5/5 [=======================

5/5 [==============================] - 0s 4ms/step - loss: 1912.7290 - val_loss: 1841.3512
Epoch 1751/10000
5/5 [==============================] - 0s 4ms/step - loss: 1813.0555 - val_loss: 1844.6782
Epoch 1752/10000
5/5 [==============================] - 0s 4ms/step - loss: 1810.7502 - val_loss: 1843.7411
Epoch 1753/10000
5/5 [==============================] - 0s 4ms/step - loss: 2227.7717 - val_loss: 1803.7322
Epoch 1754/10000
5/5 [==============================] - 0s 4ms/step - loss: 1852.2574 - val_loss: 1761.8754
Epoch 1755/10000
5/5 [==============================] - 0s 4ms/step - loss: 1753.4365 - val_loss: 1575.7455
Epoch 1756/10000
5/5 [==============================] - 0s 4ms/step - loss: 1716.2501 - val_loss: 1520.9446
Epoch 1757/10000
5/5 [==============================] - 0s 4ms/step - loss: 1922.7317 - val_loss: 1641.2650
Epoch 1758/10000
5/5 [==============================] - 0s 4ms/step - loss: 1868.7600 - val_loss: 1836.8214
Epoch 1759/10000
5/5 [=======================

In [220]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    for i in range(len(test_target)):
        prediction = model.predict(test_input.iloc[i:i+1])
        error = np.abs(prediction - test_target[start_target + i])
        errors.append(error)
        error_percent.append(error / test_target[start_target + i])
        print(f"Month-{i + 1} - Error: {error}")
    
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [221]:
errors, mae, mape = mae_mape_calculator(trained_model, test_input, test_target, 168)

1/1 [==============================] - 0s 37ms/step
Month-1 - Error: [[131.69586]]
1/1 [==============================] - 0s 14ms/step
Month-2 - Error: [[50.500793]]
1/1 [==============================] - 0s 13ms/step
Month-3 - Error: [[82.66272]]
1/1 [==============================] - 0s 13ms/step
Month-4 - Error: [[85.786316]]
1/1 [==============================] - 0s 13ms/step
Month-5 - Error: [[86.50116]]
1/1 [==============================] - 0s 13ms/step
Month-6 - Error: [[19.67041]]
1/1 [==============================] - 0s 13ms/step
Month-7 - Error: [[0.7788391]]
1/1 [==============================] - 0s 13ms/step
Month-8 - Error: [[37.348297]]
1/1 [==============================] - 0s 13ms/step
Month-9 - Error: [[74.845]]
1/1 [==============================] - 0s 13ms/step
Month-10 - Error: [[63.26761]]
1/1 [==============================] - 0s 13ms/step
Month-11 - Error: [[132.16245]]
1/1 [==============================] - 0s 13ms/step
Month-12 - Error: [[77.28183]]
1/1 [====

In [222]:
display(mae)
display(mape)

65.75104

0.16213018

In [223]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [224]:
errors, mae, mape = year_mae_mape_calculator(trained_model, test_input, test_target, 168)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 13ms/step
Ano-0: |Prediction[[5779.3877]] - Target[4938.444]| =  Error: [[840.94385]]
1/1 [==============================] - 0s 13ms/step
Ano-0: |Prediction[[6291.9253]] - Target[5580.459999999999]| =  Error: [[711.46533]]
1/1 [==============================] - 0s 13ms/step
Ano-11: |Prediction[[6416.8296]] - Target[6223.561000000001]| =  Error: [[193.26855]]


[array([[840.94385]], dtype=float32),
 array([[711.46533]], dtype=float32),
 array([[193.26855]], dtype=float32)]

581.8926

0.10961059